In [39]:
import tensorflow as tf
from tensorflow import keras

import infer_tools

from importlib import reload
reload(infer_tools);

import numpy as np
import pandas as pd

import sys
sys.path.append("../")
import utils

import json

In [7]:
VEHICLE_ID = "commercial_3-1_98"

In [9]:
model = keras.models.load_model("../../04_nn_models/ind_%s"%VEHICLE_ID)

2023-01-29 14:58:15.431095: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-29 14:58:15.434476: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-29 14:58:15.434609: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-29 14:58:15.435333: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [19]:
settings = {
    "min": 18220,
    "max": 46800,
    "mean": 0.59,
    "std": 0.35302
}

PARKING_ID_LIST = [i for i in range(1059, 1186+1)]

In [22]:
test_t = np.arange(14460, 50400+1, 60)
test_t = utils.normalize(test_t, settings["min"], settings["max"])

In [18]:
oracle = pd.read_csv("../../02_data/oracle.csv")

In [24]:
plist=[i for i in range(1059,1186+1)]
plist.remove(1148)
parking_testset = None
for parking in plist:
    id_encoding = utils.one_hot_encoder([parking]*len(test_t), PARKING_ID_LIST)
    test_d = np.hstack([id_encoding, test_t.reshape(len(test_t), 1)])
    if parking_testset is None:
        parking_testset = test_d
    else:
        parking_testset = np.vstack([parking_testset, test_d])

In [27]:
with open("../../02_data/known_parkings.json", "r") as f:
    known_parkings_by_vehicles = json.load(f)
with open("../../02_data/known_moving_times.json", "r") as f:
    known_moving_times = json.load(f)
    
true_parkings = known_parkings_by_vehicles[VEHICLE_ID]
true_time = known_moving_times[VEHICLE_ID]

In [31]:
true_parkings

[1087, 1111, 1123, 1139, 1140, 1145, 1147, 1178]

In [40]:
infer_tools.evaluate_performance(model, oracle, parking_testset, test_t,
                                 settings, true_parkings, true_time)

(1.0, 519.5)